# Stability Results

A notebook which plots the Benchmark scores for GAM and then plots GAM for the clusters found by Markov Stability. 

## Google Drive 

In [1]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/My Drive/Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Libraries

In [2]:
# standard libraries
import numpy as np
import pandas as pd

In [3]:
%%capture
# geopandas install
import os
!curl -L http://download.osgeo.org/libspatialindex/spatialindex-src-1.8.5.tar.gz | tar xz
os.chdir('/content/spatialindex-src-1.8.5')
!./configure
!make
!make install
!ldconfig
!pip install descartes
!pip install rtree
!pip install geopandas

# geopandas import
import geopandas as gpd

In [4]:
%%capture
# bokeh intstall
#!pip install bokeh

# bokeh import
from bokeh.models import HoverTool, ColumnDataSource, LinearAxis, Range1d, LogAxis
from bokeh.plotting import figure, output_file, show

In [5]:
# allows visualisation in notebook
from bokeh.io import output_notebook
from bokeh.resources import INLINE
output_notebook(INLINE)

In [6]:
# import classes
import sys
sys.path.append(root + 'Classes')
from Stability_class import Stability
from GAM_class import GAM
from Benchmark_class import Benchmark

## Load Data

Load in MSOA data.

In [7]:
MSOAs = gpd.read_file(root + 'MSOAs/MSOAs.shp')
print('Shape: ',MSOAs.shape)
MSOAs.head()

Shape:  (6790, 6)


,msoa11cd,msoa11nm,st_areasha,pop,con_trust,geometry
0,E02000001,City of London 001,2.983633e+06,6031.0,0,"POLYGON ((-0.09276 51.52139, -0.08813 51.51941..."
1,E02000002,Barking and Dagenham 001,2.091907e+06,7131.0,0,"POLYGON ((0.14112 51.58054, 0.13788 51.57812, ..."
2,E02000003,Barking and Dagenham 002,2.122216e+06,10437.0,0,"POLYGON ((0.14838 51.58075, 0.14698 51.57568, ..."
3,E02000004,Barking and Dagenham 003,2.569470e+06,6393.0,0,"POLYGON ((0.19018 51.55268, 0.18600 51.54753, ..."
4,E02000005,Barking and Dagenham 004,1.111109e+06,9116.0,0,"POLYGON ((0.15043 51.56561, 0.14998 51.56138, ..."


Loads in stability data by creating an instance of Stability class.

In [8]:
stability_data = Stability(root + 'Stability Data/longrun.mat')

# N x T array of cluster labels.
C = stability_data.C
# Array of number of communities.
k = stability_data.k
# Array of Markov times
times = stability_data.t
# Array of Variation of Information.
VI = stability_data.VI

## Benchmark Plot

Creates an instance of the Benchmark class for the MSOA data.

In [9]:
benchmark = Benchmark(MSOAs)

Obtains mu an array of expected benchmark scores for GAM for method choice from polygon, k-nearest neighbors and $\epsilon$-ball.

In [10]:
%%time
# Calculates the expected GAM
k_range = np.arange(min(k),max(k)) # gets range of k
benchmark.get_mu(k_range)
# Obtains mu
mu = benchmark.mu

What method 'poly', 'ball' or 'k'?
k
What parameter?
6
CPU times: user 2.28 s, sys: 155 ms, total: 2.43 s
Wall time: 34.6 s


Obtains benchmark score for random clusterings.

In [11]:
%%time
# only want to find sample for unique k once
samples, unique_k = benchmark.samples(root + 'Stability Data/longrun.mat')

CPU times: user 3min 34s, sys: 51.6 ms, total: 3min 34s
Wall time: 3min 34s


Uses Bokeh to plot the benchmark scores.

In [12]:
# root to name and store html file
output_file(root + 'Plots/K/K_benchmark_plot.html',mode='inline')

# change datatype to allow hover functionality
mu_source = ColumnDataSource(data=dict(
    k=k_range,
    GAM=mu,
))

sam_source = ColumnDataSource(data=dict(
    k=unique_k,
    GAM=samples,
))


# create figure
plot = figure(title='Benchmark Scores',
              x_axis_label = 'Number of communities',
              y_axis_label = 'GAM Score',
              plot_height=600,
              plot_width=800)

plot.grid.grid_line_alpha = 0.3

# make plots
plot.line('k','GAM', line_width=2,legend_label='Expected score',source=mu_source)
plot.scatter('k','GAM',marker = 'x',line_color='red',size=6,legend_label='Random Sample',source=sam_source)

# add hovertool
hover = HoverTool(tooltips=[("k","@k"),("GAM Score","@GAM")])
plot.add_tools(hover)

show(plot)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
S = benchmark.S 
print(S)

0.16666666666666666


In [14]:
benchmark.get_sigma(k_range)
sigma = benchmark.sigma

/content/drive/My Drive/Project/Classes/Benchmark_class.py:76: UserWarning: Possible floating point arithemetic error.
  warn('Possible floating point arithemetic error.')
/content/drive/My Drive/Project/Classes/Benchmark_class.py:79: RuntimeWarning: invalid value encountered in sqrt
  self.sigma = np.sqrt(c_k*GS)


## Stability Data Plot

Get GAM scores across time.

In [15]:
%%time
# get stability dataframe
stability_df = stability_data.cluster_df(MSOAs,'all')

# get GAM scores across time
scores = GAM(stability_df).GAM_scores()

What method 'poly', 'ball' or 'k'?
k
What parameter?
6
CPU times: user 7min 43s, sys: 147 ms, total: 7min 43s
Wall time: 10min 52s


Plot, Variation of Information, number of communities and GAM scores across Markov Time.

In [16]:
def GAM_plot(times,k ,scores,filename,y_label,title='Markov Stability Plot'):
    # root to name and store html file
    output_file(root + 'Plots/K/' + filename,mode='inline')

    # change datatype to allow hover functionality
    k_source = ColumnDataSource(data=dict(
        time=times,
        k=k,
    ))

    GAM_source = ColumnDataSource(data=dict(
        time=times,
        GAM=scores
    ))

    # create figure
    plot = figure(title=title,
                  x_axis_label='Markov Time',
                  x_axis_type='log',
                  y_axis_type='log',
                  y_axis_label = y_label,
                  y_range=(np.min(scores)*0.98,np.max(scores)*1.02),
                  plot_height=400,
                  plot_width=800)

    plot.yaxis.axis_label_text_color = 'blue'

    # setting second y axis range name and range
    plot.extra_y_ranges = {'clusters': Range1d(np.min(k)*0.98,np.max(k)*1.02)}

    # adding the second axis to the plot.  
    plot.add_layout(LogAxis(y_range_name='clusters',axis_label='Number of Communities',axis_label_text_color='red'), 'right')

    # make plots
    plot1 = plot.line('time','GAM',source=GAM_source)
    plot.add_tools(HoverTool(renderers=[plot1],tooltips=[('Markov time: ','@time'),(y_label + ': ','@GAM')],mode='vline'))
    plot2 = plot.line('time','k',line_color='red',y_range_name='clusters',source=k_source)
    plot.add_tools(HoverTool(renderers=[plot2],tooltips=[('Markov time: ','@time'),('Number of Communities: ','@k')],mode='vline'))

    show(plot)

GAM score vs Markov Time.

In [17]:
GAM_plot(times,k, scores,'K_GAM_score.html','GAM Score')

Output hidden; open in https://colab.research.google.com to view.

Find benchmark scores and use to adjust GAM score.

In [18]:
# get benchmark scores
benchmark.get_mu(k)
bench_scores = benchmark.mu

# adjust by minus
adj_minus = scores - bench_scores

# adjust by divide
adj_divide = scores/bench_scores

# relative
adj_relative = adj_minus/ bench_scores

GAM and mu across time.

In [19]:
title = 'GAM and mu across Markov Time'
y_label = 'GAM Score' 

# root to name and store html file
output_file(root + 'Plots/K/K_mu_GAM_plot.html',mode='inline')

# change datatype to allow hover functionality
GAM_source = ColumnDataSource(data=dict(
    time=times,
    GAM=scores
))

Bench_source = ColumnDataSource(data=dict(
    time=times,
    GAM=bench_scores
))

# create figure
plot = figure(title=title,
              x_axis_label='Markov Time',
              x_axis_type='log',
              y_axis_label = y_label,
              y_range=(np.min(scores)*0.98,np.max(scores)*1.02),
              plot_height=400,
              plot_width=800)

plot.yaxis.axis_label_text_color = 'blue'

# setting second y axis range name and range
plot.extra_y_ranges = {'clusters': Range1d(np.min(bench_scores)*0.98,np.max(bench_scores)*1.02)}

# adding the second axis to the plot.  
plot.add_layout(LinearAxis(y_range_name='clusters',axis_label='Benchmark Score',axis_label_text_color='red'), 'right')

# make plots
plot1 = plot.line('time','GAM',source=GAM_source)
plot.add_tools(HoverTool(renderers=[plot1],tooltips=[('Markov time: ','@time'),(y_label + ': ','@GAM')],mode='vline'))
plot2 = plot.line('time','GAM',line_color='red',y_range_name='clusters',source=Bench_source)
plot.add_tools(HoverTool(renderers=[plot2],tooltips=[('Markov time: ','@time'),('Benchmark Score: ','@GAM')],mode='vline'))

show(plot)

Output hidden; open in https://colab.research.google.com to view.

$\mu$ across time.

In [20]:
GAM_plot(times, k, bench_scores,'K_GAM_bench.html','Bench Score')

Output hidden; open in https://colab.research.google.com to view.

GAM score - $\mu$ vs Markov Time.

In [21]:
GAM_plot(times,k, adj_minus,'K_GAM_score_minus.html','GAM Score - $\mu$')

Output hidden; open in https://colab.research.google.com to view.

GAM score - $\mu$ vs Markov Time subplot.

In [22]:
start = 100
end = -70
GAM_plot(times[start:end],k[start:end], adj_minus[start:end],'K_GAM_score_minus_subplot.html','GAM Score - $\mu$')

Output hidden; open in https://colab.research.google.com to view.

GAM score/$\mu$ vs Markov Time.

In [23]:
GAM_plot(times,k, adj_divide,'K_GAM_score_divide.html','GAM Score /$mu$')

Output hidden; open in https://colab.research.google.com to view.

GAM score/$\mu$ vs Markov Time subplot.

In [24]:
GAM_plot(times[start:end],k[start:end], adj_divide[start:end],'K_GAM_score_divide_subplot.html','GAM Score /$mu$')

Output hidden; open in https://colab.research.google.com to view.

$\frac{GAM - \mu}{\mu}$ vs Markov Time subplot.

In [25]:
GAM_plot(times, k, adj_relative,'K_GAM_relative.html','Realtive Score')

Output hidden; open in https://colab.research.google.com to view.